In [31]:
# Constructs a quantum circuit g with parameters θ, then differentiates the recursive algorithm given in Section 5.1 of https://arxiv.org/abs/1112.2184 to obtain the gradient of p_θ(x) wrt θ, where x is a measurement of g|0>. The differentiation takes polynomial time due to memoization.
# We then compare our results to the finite difference gradient

using Yao, FLOYao
using LinearAlgebra

nq = 10 #Number of qubits
layers = 2 #Number of brick-wall layers in the circuit
g = chain(nq)
for _ in 1:layers
    for i in 1:nq-1
        push!(g, rot(kron(nq, i => X, i+1 => X), 0.)) #Nearest-neighbor XX rotation gates
    end
    for i in 1:nq-1
        push!(g, rot(kron(nq, i => X, i+1 => Y), 0.)) #Nearest-neighbor XY rotation gates
    end
    for i in 1:nq
        push!(g, put(nq, i => Rz(0.))) #Single qubit Z rotation gates
    end
end

#Set g to have random parameters
p = rand(nparameters(g)).*2π
dispatch!(g, p)
nparams = nparameters(g)
dim = 2*nq
println("number of parameters: ", nparams)

⊗ = kron

function covariance_matrix(reg::MajoranaReg)
    nq = nqubits(reg)
    G = I(nq) ⊗ [0 1; -1 0]
    return reg.state * G * reg.state'
end

function majoranaindices2kron(nq, i, j) #Returns γ_iγ_j, assuming that i≠j
    p = []
    c = (i % 2 == j % 2) ? 1 : -1
    a = min(i, j)
    b = max(i, j)
    first = (a+1) ÷ 2 
    last = (b+1) ÷ 2 
    if first == last #This means i=j-1 and j is even
        c = 1
        push!(p, first => Z)
    else
        if i % 2 == 0
            push!(p, first => X)
            c *= 1
        else
            push!(p, first => Y)
            c *= -1
        end
        for k in first+1:last-1
            push!(p, k => Z)
            c *= -1
        end
        if j % 2 == 0
            push!(p, last => Y)
        else
            push!(p, last => X)
        end
    end
    if i > j
        c *= -1
    end
    return c*kron(nq, p...)
end

function majorana_commutator(nq, i, j) #Returns [γ_i,γ_j]=2γ_iγ_j, due to the anti-commutation of Majorana operators. It needs to be an 'Add' object so that the Yao.expect' function can take it in as input.
    return Add(majoranaindices2kron(nq, i, j)) 
end

function update!(reg::MajoranaReg, theta, b, temp_m, temp_grad_m, cur_m, cur_grad_m, probabilities, grad_probabilities) #Evolves all matrices and probabilities and gradients by nq steps, in-place. This method is slow but is definitely correct. I used this to check that my more optimal function was outputting the correct thing.
    t_tot = 0
    for i in 1:nq
        t = time()
        if i > 1
            cur_m = deepcopy(temp_m)
            cur_grad_m = deepcopy(temp_grad_m)
            cur_prob = deepcopy(probabilities[i-1])
            cur_grad_prob = deepcopy(grad_probabilities[i-1, :])
            ni = b[i-1]
            for p in 1:dim
                for q in p+1:dim
                    temp_grad_m[p,q] .-= (-1)^ni * ((-cur_grad_prob * cur_m[2*(i-1)-1,p] * cur_m[2*(i-1),q]) .+ (cur_prob * (cur_grad_m[2*(i-1)-1,p]*cur_m[2*(i-1),q] .+ cur_m[2*(i-1)-1,p] * cur_grad_m[2*(i-1),q]))) / (2*cur_prob^2)
                    temp_grad_m[p,q] .+= (-1)^ni * ((-cur_grad_prob * cur_m[2*(i-1)-1,q] * cur_m[2*(i-1),p]) .+ (cur_prob * (cur_grad_m[2*(i-1)-1,q]*cur_m[2*(i-1),p] .+ cur_m[2*(i-1)-1,q] * cur_grad_m[2*(i-1),p]))) / (2*cur_prob^2)
                    temp_grad_m[q,p] = -temp_grad_m[p,q] 
                end
            end
            for p in 1:dim
                temp_grad_m[p,p] = zeros(nparams)
            end
            for p in 1:dim
                for q in p+1:dim
                    temp_m[p,q] -= (-1)^ni * (cur_m[2*(i-1)-1,p] * cur_m[2*(i-1),q]) / (2*cur_prob)
                    temp_m[p,q] += (-1)^ni * (cur_m[2*(i-1)-1,q] * cur_m[2*(i-1),p]) / (2*cur_prob)
                    temp_m[q,p] = -temp_m[p,q]
                end
            end
            for p in 1:dim
                temp_m[p,p] = 0.0
            end
            ni = b[i]
            probabilities[i] = (1+(-1)^ni * temp_m[2*i-1, 2*i]) / 2
            grad_probabilities[i, :] = (-1)^ni * temp_grad_m[2*i-1, 2*i] / 2
        else
            dispatch!(g, theta)
            temp_m = covariance_matrix(apply(reg, g))
            ni = b[i]
            probabilities[i] = (1+(-1)^ni * temp_m[2*i-1, 2*i]) / 2
            for p in 1:dim
                for q in p+1:dim
                    ham = majorana_commutator(nq, p, q)
                    temp_grad_m[p,q] = expect'(ham, reg => g)[2]
                    temp_grad_m[q,p] = -temp_grad_m[p,q]
                end
            end
            for p in 1:dim
                temp_grad_m[p,p] = zeros(nparams)
            end
            grad_probabilities[i, :] = (-1)^ni * temp_grad_m[2*i-1, 2*i] / 2
        end
        diff = time() - t
        t_tot += diff
        println("iteration $i: $diff")
    end
    println("total time: $t_tot")
end 

function update_opt!(reg::MajoranaReg, theta, b, temp_m, temp_grad_m, probabilities, grad_probabilities) #Evolves all matrices and probabilities and gradients by nq steps, in-place and optimally
    t_tot = 0
    for i in 1:nq
        t = time()
        if i > 1
            ni = b[i-1]
            cur_prob = probabilities[i-1]
            cur_grad_prob = grad_probabilities[:, i-1]
            cur_prefactor = (-1)^ni / (2*cur_prob)
            cur_grad_prefactor = (-1)^ni / (2*cur_prob^2)
            for p in 2*(i-1)+1:dim
                for q in p+1:dim
                    temp_grad_m[:,p,q] .-= cur_grad_prefactor * ((-cur_grad_prob * temp_m[2*(i-1)-1,p] * temp_m[2*(i-1),q]) .+ (cur_prob * (temp_grad_m[:,2*(i-1)-1,p]*temp_m[2*(i-1),q] .+ temp_m[2*(i-1)-1,p] * temp_grad_m[:,2*(i-1),q])))
                    temp_grad_m[:,p,q] .+= cur_grad_prefactor * ((-cur_grad_prob * temp_m[2*(i-1)-1,q] * temp_m[2*(i-1),p]) .+ (cur_prob * (temp_grad_m[:,2*(i-1)-1,q]*temp_m[2*(i-1),p] .+ temp_m[2*(i-1)-1,q] * temp_grad_m[:,2*(i-1),p])))
                end
            end
            for p in 2*(i-1)+1:dim
                for q in p+1:dim
                    temp_m[p,q] -= cur_prefactor * (temp_m[2*(i-1)-1,p] * temp_m[2*(i-1),q])
                    temp_m[p,q] += cur_prefactor * (temp_m[2*(i-1)-1,q] * temp_m[2*(i-1),p])
                end
            end
            ni = b[i]
            probabilities[i] = (1+(-1)^ni * temp_m[2*i-1, 2*i]) / 2
            grad_probabilities[:, i] = (-1)^ni * temp_grad_m[:,2*i-1, 2*i] / 2
        else
            dispatch!(g, theta)
            temp_m = covariance_matrix(apply(reg, g))
            ni = b[i]
            probabilities[i] = (1+(-1)^ni * temp_m[2*i-1, 2*i]) / 2
            for p in 1:dim
                for q in p+1:dim
                    ham = majorana_commutator(nq, p, q)
                    temp_grad_m[:,p,q] = expect'(ham, reg => g)[2]
                end
            end
            grad_probabilities[:, i] = (-1)^ni * temp_grad_m[:,2*i-1, 2*i] / 2
        end
        diff = time() - t
        t_tot += diff
        println("iteration $i: $diff")
    end
    println("total time: $t_tot")
end

function log_grad(reg::MajoranaReg, theta, b, temp_m, temp_grad_m, cur_m, cur_grad_m, probabilities, grad_probabilities) #Returns ∇_θlog(p_θ(b)), evaluated at 'theta' (parameters of circuit) and 'b' (measurement result); 'reg' is the initial register and must be of type MajoranaReg (e.g. FLOYao.zero_state(nq))
    nq = nqubits(reg)
    update!(reg, theta, b, temp_m, temp_grad_m, cur_m, cur_grad_m, probabilities, grad_probabilities)
    s = zeros(length(theta))
    for i in 1:nq
        s += grad_probabilities[i, :] / probabilities[i]
    end
    basic_prob = probabilities
    return basic_prob, s
end

function log_grad_opt(reg::MajoranaReg, theta, b, temp_m, temp_grad_m, probabilities, grad_probabilities) #Returns ∇_θlog(p_θ(b)), evaluated at 'theta' (parameters of circuit) and 'b' (measurement result); 'reg' is the initial register and must be of type MajoranaReg (e.g. FLOYao.zero_state(nq)). This uses the optimal updating function which is more efficient but still outputs the same thing as the original update! function.
    update_opt!(reg, theta, b, temp_m, temp_grad_m, probabilities, grad_probabilities)
    s = zeros(length(theta))
    for i in 1:nq
        s += grad_probabilities[:, i] / probabilities[i]
    end
    optimized_prob = probabilities
    return optimized_prob, s
end

reg = apply(FLOYao.zero_state(nq), g)
bitstr = measure(reg, nshots = 1)[1] #Random measurement of g|0>
println("measured outcome: $bitstr")
println("probability of measuring the above outcome: ", FLOYao.bitstring_probability(reg, bitstr)) #Uses FLOYao.bitstring_probability(reg, bitstr) which is known to be correct. We check this number against our algorithm output, to verify correctness.

T = Float64 #Can also be BigFloat, may experiment with other data types later
println("data type used in calculations: $T") 
println("note: the time (μs) taken for 'iteration i' refers to the time required for the algorithm to compute p_θ(x_i|x_1,...x_{i-1}) and ∇_θ(p_θ(x_i|x_1,...x_{i-1}))")

#Initializing temporary matrices and vectors used in the unoptimal version of the algorithm.
temp_m = Matrix{T}(undef, dim, dim)
temp_grad_m = Matrix{Vector{T}}(undef, dim, dim)
cur_m = Matrix{T}(undef, dim, dim)
cur_grad_m = Matrix{Vector{T}}(undef, dim, dim)
probabilities = Vector{T}(undef, nq)
grad_probabilities = Matrix{T}(undef, nq, nparams)

basic_prob, basic = log_grad(FLOYao.zero_state(nq), p, bitstr, temp_m, temp_grad_m, cur_m, cur_grad_m, probabilities, grad_probabilities)
println("The ith entry in the following vector is p_θ(x_i|x_1,...x_{i-1})")
println(basic_prob)
println("the product of all entries in the above vector, should match the earlier probability computed using FLOYao.bitstring_probability: ", prod(basic_prob))
println("The following vector is ∇_θ(log(p_θ(x))), evaluated at x = measured outcome")
println(basic)

number of parameters: 56
measured outcome: 0110011000 ₍₂₎
probability of measuring the above outcome: 0.0002616637903138677054820166784700677419070212165444083615012690473173857106497774
data type used in calculations: Float64
note: the time (μs) taken for 'iteration i' refers to the time required for the algorithm to compute p_θ(x_i|x_1,...x_{i-1}) and ∇_θ(p_θ(x_i|x_1,...x_{i-1}))
iteration 1: 0.31081390380859375
iteration 2: 0.027170896530151367
iteration 3: 0.027393102645874023
iteration 4: 0.03413701057434082
iteration 5: 0.02690601348876953
iteration 6: 0.02078413963317871
iteration 7: 0.021872997283935547
iteration 8: 0.030344009399414062
iteration 9: 0.02223801612854004
iteration 10: 0.0220029354095459
total time: 0.5436630249023438
The ith entry in the following vector is p_θ(x_i|x_1,...x_{i-1})
[0.3559703909266713, 0.635724784341615, 0.02725358381354659, 0.49950182735153437, 0.505037572999305, 0.6938682841194418, 0.7358149317075242, 0.338125530434357, 0.9742100869212049, 1.0]


In [32]:
#Initializing temporary matrices and vectors for the optimized version of the algorithm.
temp_m = Matrix{T}(undef, dim, dim)
temp_grad_m = Array{T}(undef, nparams, dim, dim)
probabilities = Vector{T}(undef, nq)
grad_probabilities = Matrix{T}(undef, nparams, nq)

#Calling the optimized version of the algorithm. 'optimized_prob' represents the vector with ith entry p_θ(x_i|x_1,...x_{i-1}) and 'optimized' is ∇_θ(log(p_θ(x))).
optimized_prob, optimized = log_grad_opt(FLOYao.zero_state(nq), p, bitstr, temp_m, temp_grad_m, probabilities, grad_probabilities)

#Checking that the output of the optimized algorithm still matches the output of the unoptimized algorithm.
println("probabilities equal? ", basic_prob == optimized_prob)
println("grad(log p) equal? ", basic == optimized)
println("The following vector is ∇_θ(log(p_θ(x))), evaluated at x = measured outcome, as outputted by the optimized version of the algorithm.")
println(optimized)

iteration 1: 0.03325700759887695
iteration 2: 0.012118101119995117
iteration 3: 0.008212804794311523
iteration 4: 0.006367921829223633
iteration 5: 0.008416891098022461
iteration 6: 0.003178119659423828
iteration 7: 0.0020661354064941406
iteration 8: 0.0011591911315917969
iteration 9: 0.0004830360412597656
iteration 10: 0.00011420249938964844
total time: 0.07537341117858887
probabilities equal? true
grad(log p) equal? true
The following vector is ∇_θ(log(p_θ(x))), evaluated at x = measured outcome, as outputted by the optimized version of the algorithm.
[-0.11723147692605644, 0.5278587143317579, 2.1088493964730843, 0.13601704946657878, -1.3760787454354042, -0.5816743066899707, -0.2744218849549109, -0.1666610294427909, -0.24314332291634833, -0.8939293912640738, -0.29038814290154435, -0.19136976880901807, 0.024156764460646806, -0.613093257384518, -0.36879866077965484, -0.10123751788856125, -0.7053882899036116, 0.22973712556915368, -0.514062023628584, 1.0905344638530237, 0.029484916347502

In [33]:
#Comparison with finite difference method
using LinearAlgebra

function prob(theta, x) #Outputs p_θ(x), the probability of measuring an outcome of 'x' for the state g|0> where the parameters of g are set to 'theta'
    circuit = dispatch(g, theta)
    r = apply(FLOYao.zero_state(nq), circuit)
    return FLOYao.bitstring_probability(r, x)
end

eps_default = 1e-8
function fe_grad_prob(theta, x, eps = eps_default) #Computes the finite-difference approximation for ∇_θ(log(p_θ(x))), evaluated at 'x'. I mainly used this to verify the correctness of the algorithm
    temp_params = copy(theta)
    fe_grad = Vector{Float64}(undef, length(theta))
    for i in 1:nparameters(g)
        temp_params[i] += eps
        plus = log(prob(temp_params, x))
        temp_params[i] -= 2*eps
        minus = log(prob(temp_params, x))
        fe_grad[i] = (plus - minus) / (2*eps)
        temp_params[i] += eps #Resetting temp_params[i] back to original value
    end
    fe_grad
end

println("algorithm output for ∇_θ(log(p_θ(x))), should be exact")
println(basic)
fe = fe_grad_prob(p, bitstr)
println("finite difference approximation to ∇_θ(log(p_θ(x)))")
println(fe)
println("l2 distance between algorithm output and finite difference approximation: ")
println(norm(basic - fe))

algorithm output for ∇_θ(log(p_θ(x))), should be exact
[-0.11723147692605644, 0.5278587143317579, 2.1088493964730843, 0.13601704946657878, -1.3760787454354042, -0.5816743066899707, -0.2744218849549109, -0.1666610294427909, -0.24314332291634833, -0.8939293912640738, -0.29038814290154435, -0.19136976880901807, 0.024156764460646806, -0.613093257384518, -0.36879866077965484, -0.10123751788856125, -0.7053882899036116, 0.22973712556915368, -0.514062023628584, 1.0905344638530237, 0.029484916347502162, 0.2245991292958372, -0.4396276393745268, -1.0354176148474312, -0.7445394196110664, -0.47262418648201326, -0.0506807252746101, -0.017117962290154696, 1.3305797063560811, 0.15363559352580522, 0.722943342754972, 0.23591533086365088, -1.2026839189277292, -0.04296362402798328, 0.22971396786837334, 1.2450872368979011, -0.05219589829292485, -1.3111447092712178, -0.7532087668504361, 5.677904825330657, 0.5450927043621708, -0.7764104316902812, 0.5066640657175019, 0.5134972089815872, -0.3693256606919513, -